In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

In [4]:
sum_of_outputs = 0
for sqrt_value in generator:
    sum_of_outputs += sqrt_value

print(sum_of_outputs)

8.382332347441762


In [5]:
limit = 13
generator = square_root_generator(limit)

count = 0
for sqrt_value in generator:
    count += 1
    if count == 13:
        print(sqrt_value)

3.605551275463989


In [18]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

In [9]:
sum_of_ages = 0

for person in people_1():
    sum_of_ages += person['Age']

In [10]:
print(sum_of_ages)

140


In [11]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


In [12]:
def people_3():
    for person in people_1():
        yield person
    for person in people_2():
        yield person

In [13]:
sum_of_ages = 0

for person in people_3():
    sum_of_ages += person['Age']

print(sum_of_ages)

353


In [19]:
import dlt

# define the connection to load to
# we now use duckdb, but you can switch to BigQuery later
generators_pipeline = dlt.pipeline(destination='duckdb',
								  dataset_name='generators')

# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
                        table_name='people_table',
                        write_disposition='replace')

# the outcome metadata is returned by the load and we can inspect it by printing it
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/workshop_1_data_ingesting_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708196329.731613 is LOADED and contains no failed jobs


In [21]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables:')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table below:")

people = conn.sql("SELECT * FROM people_table").df()
people

Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
│ people_table        │
│ stream_download     │
└─────────────────────┘




 people_table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708196329.731613,sGE+S5amsscX7A
1,2,Person_2,27,City_A,1708196329.731613,6WFo1IGq06Pfow
2,3,Person_3,28,City_A,1708196329.731613,41OCH5SVSD1geA
3,4,Person_4,29,City_A,1708196329.731613,PQA07MUVCiffhA
4,5,Person_5,30,City_A,1708196329.731613,bsr6tYfKxBoe/w


In [22]:
people['age'].sum()

140

In [23]:
# appending to existing table
info = generators_pipeline.run(people_2(),
                        table_name='people_table',
                        write_disposition='append')

# the outcome metadata is returned by the load and we can inspect it by printing it
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/workshop_1_data_ingesting_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708196506.97739 is LOADED and contains no failed jobs


In [24]:
print("\n\n\n people_table below:")

people = conn.sql("SELECT * FROM people_table").df()
people




 people_table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708196329.731613,sGE+S5amsscX7A,None
1,2,Person_2,27,City_A,1708196329.731613,6WFo1IGq06Pfow,None
2,3,Person_3,28,City_A,1708196329.731613,41OCH5SVSD1geA,None
3,4,Person_4,29,City_A,1708196329.731613,PQA07MUVCiffhA,None
4,5,Person_5,30,City_A,1708196329.731613,bsr6tYfKxBoe/w,None
5,3,Person_3,33,City_B,1708196506.97739,tbLFND8kIh9a9g,Job_3
6,4,Person_4,34,City_B,1708196506.97739,07VyB9A3SX1urQ,Job_4
7,5,Person_5,35,City_B,1708196506.97739,3OtBSzdmJsiKRA,Job_5
8,6,Person_6,36,City_B,1708196506.97739,KaGHZ7Bx8vgCKA,Job_6
9,7,Person_7,37,City_B,1708196506.97739,HRkiBQr5y2CHyw,Job_7


In [25]:
people['age'].sum()

353

In [27]:
# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
                        table_name='people_table_id',
                        write_disposition='replace',
                        primary_key='ID')

# the outcome metadata is returned by the load and we can inspect it by printing it
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/workshop_1_data_ingesting_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708196667.732363 is LOADED and contains no failed jobs


In [28]:
# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_2(),
                        table_name='people_table_id',
                        write_disposition='merge',
                        primary_key='ID')

# the outcome metadata is returned by the load and we can inspect it by printing it
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/jessicadesilva/Documents/GitHub/data-engineering-zoomcamp/workshop_1_data_ingesting_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708196689.851129 is LOADED and contains no failed jobs


In [29]:
people_id = conn.sql("SELECT * FROM people_table_id").df()

In [30]:
people_id['age'].sum()

266